In [2]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
import matplotlib.pyplot as plt
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Load the dataset
dataset = read_csv("iztech.csv", #veri setinin adı
                   parse_dates={'dt': ['Name']}, #tarih sütununun adı
                   infer_datetime_format=True, #tarih formatı
                   index_col=0, #indeks sütunu
                   na_values=['nan', '?'] #eksik değerler
                   ) #veri setini oku
dataset.fillna(0, inplace=True) #eksik değerleri 0 ile doldur
values = dataset.values #veri setini değerlere dönüştür
dataset.head(4) #veri setinin ilk 4 satırını göster

# Drop unnecessary columns
dataset.drop(columns=['WS101Max', 'WS101Min', 'WS101Count', 'WS76', 'WS76Max', 'WS76Min', 'WS76Std', 'WS76Count', 'WD28Max', 'WD28Min', 'WD28Std', 'WD28Count', 'rho10', 'rho10Max', 'rho10Std'], inplace=True) #gereksiz sütunları sil
dataset.drop(columns=['WS101Std', 'WS30', 'WS30Max', 'WS30Min', 'WS30Std', 'WS30Count', 'WD74', 'WD74Max', 'WD74Min', 'WD74Std', 'WD74Count', 'WD28'], inplace=True) #gereksiz sütunları sil
dataset = dataset[['T3', 'P2', 'RH3', 'WS101']] #sütunları yeniden düzenle
values = dataset.values #veri setini değerlere dönüştür

# Ensure all data is float
values = values.astype('float32') #veri setini float32 formatına dönüştür

# Normalize input features
scaler = MinMaxScaler(feature_range=(0, 1)) #veri setini 0-1 aralığına dönüştür
scaled = scaler.fit_transform(values) #veri setini dönüştür
scaled = pd.DataFrame(scaled) #veri setini dataframe formatına dönüştür 

def create_ts_data(dataset, lookback=1, predicted_col=3): #zaman serisi veri seti oluştur
    temp = dataset.copy() #veri setini kopyala
    temp["id"] = range(1, len(temp) + 1) #id sütunu ekle
    temp = temp.iloc[:-lookback, :] #sondan lookback kadar satırı sil
    temp.set_index('id', inplace=True) #id sütununu indeks olarak ata

    predicted_value = dataset.copy() #veri setini kopyala
    predicted_value = predicted_value.iloc[lookback:, predicted_col] #lookback kadar satırı sil
    predicted_value.columns = ["Predicted"] #sütun adını değiştir
    predicted_value = pd.DataFrame(predicted_value) #veri setini dataframe formatına dönüştür
    predicted_value["id"] = range(1, len(predicted_value) + 1) #id sütunu ekle
    predicted_value.set_index('id', inplace=True) #id sütununu indeks olarak ata

    final_df = pd.concat([temp, predicted_value], axis=1) #veri setlerini birleştir

    return final_df #veri setini dönüştür

# Create time series data with a lookback of 1
reframed_df = create_ts_data(scaled, 1, 3) #zaman serisi veri seti oluştur
reframed_df.fillna(0, inplace=True) #eksik değerleri 0 ile doldur
reframed_df.columns = ['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)'] #sütun adlarını değiştir

# Split data into train and test sets
values = reframed_df.values #veri setini değerlere dönüştür
training_sample = int(len(dataset) * 0.7) #veri setinin %70'ini eğitim için kullan
train = values[:training_sample, :] #veri setinin %70'ini eğitim için kullan
test = values[training_sample:, :] #veri setinin %30'unu test için kullan

# Split data into input and output
train_X, train_y = train[:, :-1], train[:, -1] #veri setini girdi ve çıktı olarak ayır
test_X, test_y = test[:, :-1], test[:, -1] #veri setini girdi ve çıktı olarak ayır

# Reshape input to be 3D [samples, time steps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1])) #veri setini yeniden şekillendir
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1])) #veri setini yeniden şekillendir

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) #veri setinin boyutunu göster

model_lstm = Sequential() #modeli oluştur
model_lstm.add(LSTM(75, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2]))) #modeli oluştur
model_lstm.add(LSTM(units=30, return_sequences=True)) #modeli oluştur
model_lstm.add(LSTM(units=30)) #modeli oluştur
model_lstm.add(Dense(units=1)) #modeli oluştur
model_lstm.compile(loss='mae', # loss fonksiyonu olarak ortalama mutlak hata kullan
                   optimizer='adam' #optimizasyon fonksiyonu olarak adam kullan. Adam, momentum ve RMSprop algoritmalarını birleştirir
                   ) #modeli derle

model_lstm.summary() #modelin özetini göster

lstm_history = model_lstm.fit(train_X, train_y,  #veri setini eğit
                              epochs=10,  #10 epoch eğitim yap
                              validation_data=(test_X, test_y), #veri setini test et 
                              batch_size=64,  #64 veri seti kullan
                              shuffle=False) #veri setini karıştırma
pred_y = model_lstm.predict(test_X) #veri setini tahmin et
pred_y_1 = model_lstm.predict(train_X) #veri setini tahmin et 

plt.plot(lstm_history.history['loss'], label='lstm train', color='brown') #eğitim verilerini çiz
plt.plot(lstm_history.history['val_loss'], label='lstm test', color='blue') #test verilerini çiz
plt.legend() #çizimi göster
plt.show() #çizimi göster

MSE = mean_squared_error(test_y, pred_y) #ortalama karesel hata
R2 = r2_score(test_y, pred_y) #r2 skoru 
RMSE = sqrt(mean_squared_error(test_y, pred_y)) #kök ortalama karesel hata
MAE = mean_absolute_error(test_y, pred_y) #ortalama mutlak hata

print("MSE:", MSE) #ortalama karesel hata
print("R2:", R2) #r2 skoru
print("RMSE:", RMSE) #kök ortalama karesel hata
print("MAE:", MAE)  #ortalama mutlak hata

plt.plot(test_y, label='Actual') #gerçek değerleri çiz
plt.plot(pred_y, label='Predicted') #tahmin edilen değerleri çiz
plt.legend() #çizimi göster
plt.show() #çizimi göster
 
tra = np.concatenate([train_X, test_X]) #veri setini birleştir
tes = np.concatenate([train_y, test_y]) #veri setini birleştir
fp = model_lstm.predict(tra) #veri setini tahmin et
plt.plot(tes, label='Actual') #gerçek değerleri çiz
plt.plot(fp, label='Predicted') #tahmin edilen değerleri çiz
plt.legend() #çizimi göster
plt.show() #çizimi göster

dataset_1 = dataset[['WS101']] #sütunları yeniden düzenle
dataset_1 = dataset_1[40421:57744] #veri setini yeniden düzenle
values_1 = dataset_1.values     #veri setini değerlere dönüştür

values_1 = values_1.astype('float32') #veri setini float32 formatına dönüştür

scaler_1 = MinMaxScaler(feature_range=(0, 1)) #veri setini 0-1 aralığına dönüştür
scaled_1 = scaler_1.fit_transform(values_1) #veri setini dönüştür
scaled_1 = pd.DataFrame(scaled_1) #veri setini dataframe formatına dönüştür

inversed_1 = scaler_1.inverse_transform(scaled_1) #veri setini dönüştür
print(inversed_1) #veri setini dönüştür
print(values_1) #veri setini dönüştür
print(scaled_1)     #veri setini dönüştür

test_y_inv = scaler_1.inverse_transform([test_y]) #veri setini dönüştür
test_y_inv = test_y_inv.reshape(17323, 1) #veri setini yeniden şekillendir

pred_y_inv = scaler_1.inverse_transform(pred_y)     #veri setini dönüştür
pred_y_1_inv = scaler_1.inverse_transform(pred_y_1) #veri setini dönüştür

MSE = mean_squared_error(test_y_inv, pred_y_inv) #ortalama karesel hata
R2 = r2_score(test_y_inv, pred_y_inv) #r2 skoru
RMSE = sqrt(mean_squared_error(test_y_inv, pred_y_inv)) #kök ortalama karesel hata
MAE = mean_absolute_error(test_y_inv, pred_y_inv) #ortalama mutlak hata

print("MSE (Inverse Normalized):", MSE) #ortalama karesel hata
print("R2 (Inverse Normalized):", R2) #r2 skoru
print("RMSE (Inverse Normalized):", RMSE) #kök ortalama karesel hata
print("MAE (Inverse Normalized):", MAE) #ortalama mutlak hata


ValueError: Missing column provided to 'parse_dates': 'Name'